In [8]:
import numpy as np
from keras.models import load_model
from music21 import instrument, note, chord, midi
import os
import pretty_midi
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

  Using cached pretty_midi-0.2.10-py3-none-any.whl
  Using cached mido-1.3.2-py3-none-any.whl.metadata (6.4 kB)
Using cached mido-1.3.2-py3-none-any.whl (54 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:



#function to load midi files from a folder, because of the nesting of the folders  add a root file walk so as to go through the directory 
def load_midi_files(directory):
    midi_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.mid'):
                file_path = os.path.join(root, file)
                try:
                    midi_files.append(pretty_midi.PrettyMIDI(file_path))
                except:
                    print(f"Could not parse {file_path}")
    return midi_files

In [13]:
#function to exttract the notes in a midi file so asto have data points to work with 
def extract_notes(midi):
    notes = []
    for instrument in midi.instruments:
        if not instrument.is_drum:
            for note in instrument.notes:
                notes.append((note.start, note.end, note.pitch, note.velocity))
    return notes

In [16]:




midi_files = load_midi_files('/Users/nabilomondi/Downloads/Final Project IRONHACK/MIDI FILE/lmd_matched/F/F/A')

# Function to map predictions to notes
def int_to_note_mapping(predictions, int_to_note):
    return [int_to_note[np.argmax(prediction)] for prediction in predictions]

# Function to generate music with temperature sampling
def generate_music(model, seed_sequence, length, int_to_note, temperature=1.0):
    generated = []
    current_sequence = seed_sequence
    for _ in range(length):
        prediction = model.predict(current_sequence.reshape(1, len(current_sequence), 4))[0]

        # Apply temperature sampling
        prediction = np.log(prediction + 1e-9) / temperature
        exp_preds = np.exp(prediction)
        prediction = exp_preds / np.sum(exp_preds)

        # Sample from the distribution
        next_index = np.random.choice(len(prediction), p=prediction)
        next_note = int_to_note[next_index]

        generated.append(next_note)
        next_note_array = np.zeros((1, 4))
        next_note_array[0, 0] = next_index
        current_sequence = np.vstack([current_sequence[1:], next_note_array])

    return generated

# Convert generated notes to MIDI file
def create_midi(generated_notes, output_file='generated_music.mid'):
    offset = 0
    output_notes = []

    for pattern in generated_notes:
        # Handle chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            # Handle single note
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        offset += 0.5

    midi_stream = midi.stream.Stream(output_notes)
    midi_stream.write('midi', fp=output_file)

# Load your trained model
model = load_model('/Users/nabilomondi/Documents/Finalproject/my_model.h5')

# Load your data and prepare int_to_note mapping
# Assuming notes and chords have been extracted and converted to integers
notes = notes
pitchnames = sorted(set(item for item in notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

# Assuming you have your training data sequences and the model is trained
# X = ... # your sequence data here
# seed_sequence = X[-2]  # Use the last sequence in the training data as the seed sequence

# Generate music
generated_notes = generate_music(model, seed_sequence, length=500, int_to_note=int_to_note, temperature=0.8)

# Create MIDI file
create_midi(generated_notes)


AttributeError: 'list' object has no attribute 'instruments'